![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [96]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [97]:
# Preprocess the data (replacing missing values will null)

In [98]:
cc_apps.replace('?',np.nan,inplace=True)
print(cc_apps.isna().sum())

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64


In [99]:
# Preprocess the data (Imputing)

In [100]:
cc_apps_copy=cc_apps.copy()
for col in cc_apps_copy.columns:
    if cc_apps_copy[col].dtype == 'object': 
        most_frequent =cc_apps_copy[col].mode()[0]  
        cc_apps_copy[col].fillna(most_frequent, inplace=True)
    else:
        mean_value = cc_apps_copy[col].mean() 
        cc_apps_copy[col].fillna(mean_value, inplace=True)

cc_apps_copy.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [101]:
# One-hot encoding with categorical columns only

In [102]:
cc_apps_encoded = pd.get_dummies(cc_apps_copy, drop_first=True)

In [103]:
# Prepare for modeling

In [104]:
X = cc_apps_encoded.iloc[:, :-1].values
y = cc_apps_encoded.iloc[:, [-1]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [105]:
# Perform Logistic Regression

In [106]:
logreg = LogisticRegression()


logreg.fit(rescaledX_train, y_train)

y_train_pred = logreg.predict(rescaledX_train)


print(confusion_matrix(y_train, y_train_pred))


[[207   3]
 [  3 270]]


In [107]:
params={'tol': [1e-4, 1e-3, 1e-2, 1e-1],  # Different tolerances
    'max_iter': [50, 100, 200, 300]   # Different max iterations
}
grid_model = GridSearchCV(estimator=logreg, param_grid=params, cv=5)
grid_model_result =grid_model.fit(rescaledX_train,y_train)
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_
print(f"The best score: {best_train_score} "  )
print(f"Parameters used: {best_train_params}")

The best score: 0.8384664948453608 
Parameters used: {'max_iter': 50, 'tol': 0.1}


In [108]:
# The best model accuracy

In [109]:
best_model = grid_model_result.best_estimator_
best_score =  best_model.score(rescaledX_test, y_test)
print(f"The best estimator: {best_score}")

The best estimator: 0.8019323671497585
